# Worksheet 08

Name: Yuchen Lu \
UID: U56150111

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [7]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[6.290422492034789, 4.160522895905762, 5.203719721359623, 7.175245436772379, 5.860775143209372, 4.029064272228796, 3.4454305391402684, 5.319942560591727, 7.287297915055268, 4.38523443722263]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [8]:
c2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(c2)

[9.582367903333637, 8.16932346763817, 7.6412129929905035, 8.768689562468364, 7.454546915494593, 8.533785888439626, 8.862339246316795, 7.82834802684151, 7.489810502035363, 9.235785028774712]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [9]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[9.235785028774712, 4.38523443722263, 7.287297915055268, 7.489810502035363, 5.319942560591727, 3.4454305391402684, 7.82834802684151, 8.862339246316795, 8.533785888439626, 7.454546915494593]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

We know the mean, variance, and the probability of each component.
For c1, the mean is 5, the variance is 1, and the probability is 0.5. 
For c2, the mean is 8, the variance is 1, and the probability is 0.5.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [10]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[9.235785028774712, 7.287297915055268, 7.489810502035363, 7.82834802684151, 8.862339246316795, 8.533785888439626, 7.454546915494593]
[4.38523443722263, 5.319942560591727, 3.4454305391402684]
P(C_1) = 0.7,  P(C_2) = 0.3
mean_1 = 8.098844788993981,  mean_2 = 4.383535845651542
var_1 = 0.5117928563720008,  var_2 = 0.5856339957010016


/opt/homebrew/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


The actual value for mean is 5 and 8, and the estimated value is 4.9 and 8.1. They are close to the true values. The actual value for variance is 1 and 1, and the estimated value is 0.51 and 0.58, which is not really close. The actual value for P(C_j) is 0.5 and 0.5, and the estimated value is 0.7 and 0.3, which is not as close.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [12]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  9.235785028774712
probability of observing that point if it came from cluster 0 =  0.06609901744172739
probability of observing that point if it came from cluster 1 =  8.440396437300413e-16
point =  4.38523443722263
probability of observing that point if it came from cluster 0 =  2.876467960677449e-12
probability of observing that point if it came from cluster 1 =  0.6812114823958105
point =  7.287297915055268
probability of observing that point if it came from cluster 0 =  0.22172531366291287
probability of observing that point if it came from cluster 1 =  3.1241196707493237e-06
point =  7.489810502035363
probability of observing that point if it came from cluster 0 =  0.38398374733503804
probability of observing that point if it came from cluster 1 =  5.298150496473375e-07
point =  5.319942560591727
probability of observing that point if it came from cluster 0 =  3.0893271032614335e-07
probability of observing that point if it came from cluster 1 =  0.18971760919953734
point

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [13]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]
var = [sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.6999996272566816,  P(C_2) = 0.30000037274331826
mean_1 = 8.098844114018005,  mean_2 = 4.3835420367755695
var_1 = 0.511796668480175,  var_2 = 0.5856538623438665


The mean_j is 4.4 and 8.1, which is close to the original value. The variance_j is 0.51 and 0.58, which is not close to the original value. The P(C_j) is 0.7 and 0.3, which is not close to the original value.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [16]:
prob_c0_x_new = [] # P(C_0 | X_i)
prob_c1_x_new = [] # P(C_1 | X_i)

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x_new.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x_new.append( pdf_i[1] * prob_c[1] / prob_x )

print("Old probabilities(c0): ", prob_c0_x)
print("New probabilities(c0): ", prob_c0_x_new)
print("Old probabilities(c1): ", prob_c1_x)
print("New probabilities(c1): ", prob_c1_x_new)


point =  9.235785028774712
probability of observing that point if it came from cluster 0 =  0.06610076114557013
probability of observing that point if it came from cluster 1 =  8.460528319088651e-16
point =  4.38523443722263
probability of observing that point if it came from cluster 0 =  2.877602339348398e-12
probability of observing that point if it came from cluster 1 =  0.6811883952740259
point =  7.287297915055268
probability of observing that point if it came from cluster 0 =  0.22172827844458953
probability of observing that point if it came from cluster 1 =  3.126783968530824e-06
point =  7.489810502035363
probability of observing that point if it came from cluster 0 =  0.38398554003573526
probability of observing that point if it came from cluster 1 =  5.303326598743782e-07
point =  5.319942560591727
probability of observing that point if it came from cluster 0 =  3.0900046925721927e-07
probability of observing that point if it came from cluster 1 =  0.1897308341929567
point =

There is not much difference between the updated P(C_j | X_i) and the original P(C_j | X_i).

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [25]:
assignment = []

for i in range(len(data)):
    if prob_c0_x_new[i] > prob_c1_x_new[i]:
        assignment.append(0)
    else:
        assignment.append(1)

print("Data Point       | Cluster Assignment")
for data_point, cluster in zip(data, assignment):
    print(f"{data_point} | {cluster}")

Data Point       | Cluster Assignment
9.235785028774712 | 0
4.38523443722263 | 1
7.287297915055268 | 0
7.489810502035363 | 0
5.319942560591727 | 1
3.4454305391402684 | 1
7.82834802684151 | 0
8.862339246316795 | 0
8.533785888439626 | 0
7.454546915494593 | 0


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      no       |
| A  C |      yes       |
| A  D |      yes       |
| A  E |      yes       |
| B  C |      yes       |
| B  D |      yes       |
| B  E |      yes       |
| C  D |      yes       |
| C  E |      yes       |
| D  E |      no       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

$$ \binom{N}{2} = \frac{N!}{2!(N-2)!} 
\\
 \binom{N}{2} = \frac{N(N-1)}{2}$$


Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?


$$
\text{Disagreements} = \frac{3(3-1)}{2} = 3
$$


$$
\text{Disagreements} = \frac{2(2-1)}{2} = 1
$$


$$
\text{Disagreements} = \frac{4(4-1)}{2} = 6
$$


$$
\text{Total Disagreements} = 3 + 1 + 6 = 10
$$

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?


$$
\text{Agreements} = \frac{3 \times (3-1)}{2} = 3 \\


\text{Agreements} = \frac{2 \times (2-1)}{2} = 1\\


\text{Agreements} = \frac{4 \times (4-1)}{2} = 6\\
$$
$$
So, the total number of agreements is $$\(3 + 1 + 6 = 10\).\\
$$
\frac{9 \times (9-1)}{2} = 36 \\



\text{Total Disagreements} = \text{Total Pairs} - \text{Total Agreements} = 36 - 10 = 26


e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

1. extract the data points in each distinct cluster
2. compute the number of disagreements in each cluster by comparing all pairs of points in that cluster and checking whether they agree or disagree
3. compute the total number of disagreements
